In [ ]:
import pandas as pd
import glob
import os

stock_data = pd.read_csv('/Users/paul/Desktop/financetest/combined_stock_data.csv', parse_dates=['Date'])
stock_data.set_index('Date', inplace=True)

tickers_and_currencies = pd.read_csv('/Users/paul/Desktop/financetest/fx_data/tickers_and_currencies.csv')

#load all FX rate data from the folder
fx_data = {}

fx_folder_path = '/Users/paul/Desktop/financetest/fx_data/'

fx_files = glob.glob(os.path.join(fx_folder_path, '*_to_usd.csv'))

for fx_file in fx_files:
    # Extract currency from the file name ('BRL_to_usd.csv' -> 'BRL')
    currency = os.path.basename(fx_file).split('_')[0]
    
    # Load the FX data from the CSV file
    fx_df = pd.read_csv(fx_file, parse_dates=['Date'])
    
    fx_df['FX_rate'] = pd.to_numeric(fx_df['FX_rate'], errors='coerce')
    
    # Remove duplicates
    fx_df = fx_df.drop_duplicates(subset='Date')
    
    fx_data[currency] = fx_df

for ticker, currency in zip(tickers_and_currencies['Ticker'], tickers_and_currencies['Currency']):
    if currency != 'USD': 
        stock_prices = stock_data[ticker].copy()
        
        stock_prices = pd.to_numeric(stock_prices, errors='coerce')
        
        fx_rates = fx_data[currency].copy()
        
        if 'Date' in fx_rates.columns:
            fx_rates.set_index('Date', inplace=True)
        
        # Remove duplicate dates in both datasets
        stock_prices = stock_prices[~stock_prices.index.duplicated(keep='first')]
        fx_rates = fx_rates[~fx_rates.index.duplicated(keep='first')]
        
  
        aligned_fx = fx_rates.reindex(stock_prices.index)
        
        aligned_fx['FX_rate'].fillna(method='ffill', inplace=True)
        aligned_fx['FX_rate'].fillna(method='bfill', inplace=True)
        aligned_fx['FX_rate'].fillna(1, inplace=True)
        
        # Convert to USD here: 
        stock_data.loc[stock_prices.index, ticker] = stock_prices * aligned_fx['FX_rate']

stock_data.to_csv('/Users/paul/Desktop/financetest/combined_stock_data_in_usd.csv')


Conversion completed successfully. The new CSV is saved as 'combined_stock_data_in_usd.csv'.


/var/folders/mh/fnysm72j3g53gdg3ld3708nw0000gn/T/ipykernel_91356/2390591845.py:61: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  aligned_fx['FX_rate'].fillna(method='ffill', inplace=True)
/var/folders/mh/fnysm72j3g53gdg3ld3708nw0000gn/T/ipykernel_91356/2390591845.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aligned_fx['FX_rate'].fillna(method='ffill', inplace=True)
/var/folders/mh/fnysm72j3g53gdg3ld3708nw0000gn/T/ipykernel_91356/23